In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pdb, argparse
import time, timeit
import os, glob
import copy
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from skimage import io
import numpy as np
import errno
from datetime import datetime
import  torch.nn.functional as F
from torch.autograd import Variable
from ResNet_Image import ACmix_ResNet
from yellowbrick.classifier import ROCAUC
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
import sys

In [5]:
def center_crop(img, dim):
    """Returns center cropped image
    返回居中裁剪的图像
    Args:
    img: image to be center cropped
    dim: dimensions (width, height) to be cropped 要裁剪的尺寸（宽度，高度）。
    """
    width, height = img.shape[1], img.shape[0]

    # process crop width and height for max available dimension 处理作物宽度和高度的最大可用尺寸
    crop_width = dim[0] if dim[0] < img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1] < img.shape[0] else img.shape[0]
    mid_x, mid_y = int(width / 2), int(height / 2)
    cw2, ch2 = int(crop_width / 2), int(crop_height / 2)
    crop_img = img[mid_y - ch2:mid_y + ch2, mid_x - cw2:mid_x + cw2]
    return crop_img

In [6]:
def save_model(model, his):#保存训练好的模型
    mydir = os.path.join(
        os.getcwd(),
        'log',
        datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    try:
        os.makedirs(mydir)
    except OSError as e:
        
        if e.errno != errno.EEXIST:
            
            raise  # This was not a "directory exist" error..
    torch.save({'model_state_dict': model.state_dict(), 'his': his}, os.path.join(mydir, 'checkpoint.pt'))
    print(f'model saved to {mydir}')

In [7]:
def write_resultT(fileloc, epoch, loss, accuracy):
    with open(fileloc,"a") as f:
        data = "Epoch: "+ str(epoch) + "\tTrainLoss " + str(loss) + "\tTrainAccuracy " + str(accuracy) + "\n"
        f.write(data)
def write_resultV(fileloc, epoch, loss, accuracy):
    with open(fileloc,"a") as f:
        data = "Epoch: "+ str(epoch) + "\tvalLoss " + str(loss) + "\tvalAccuracy " + str(accuracy) + "\n"
        f.write(data)

In [8]:
def load_model(ckpt_path, model, device='cuda'):#加载模型
    checkpoint = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'],strict=False)
    print(f'model load from {ckpt_path}')
    return model

In [9]:
class MyCropTransform:#自定义数据

    def __init__(self):
        pass
    def __call__(self, x):
        x = TF.crop(x, top=200, left=400, height=800, width=800)
        return x

In [10]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.  # 迭代数据
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients 梯度清零
                optimizer.zero_grad()

                # forward
                # track history if only in train # 向前传播
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss  获取模型输出并计算损失
                    # Special case for inception because in training it has an auxiliary output. In train
                    # 获取模型输出并计算损失，开始的特殊情况在训练中他有一个辅助输出
                    # 在训练模式下，通过将最终输出和辅助输出相加来计算损耗，在测试中值考虑最终输出
                    #我们通过将最终输出和辅助输出相加来计算损失
                    #   but in testing we only consider the final output. 但在测试中我们只考虑最终的输出。
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:

                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    # backward + optimize only if in training phase  反向+ 只有在训练阶段才进行优化
                    if phase == 'train':
                        loss.backward()#反向传播
                        optimizer.step()#梯度清零

                # statistics 统计数据
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            if phase == 'train':
                write_resultT("./loss/attention3Train.txt", epoch + 1, epoch_loss, epoch_acc)
            else:
                write_resultV("./loss/attention3Val.txt", epoch + 1, epoch_loss, epoch_acc)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            # deep copy the model 复制模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights 加载最好模型的权重
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


In [11]:
def set_parameter_requires_grad(model, feature_extracting):#冻结参数梯度
    if feature_extracting:
        # 冻结卷积层
        flag_m1 = 0
        #flag_m1 = 1
        
        if flag_m1:
            for param in model.parameters():
                param.requires_grad = False

In [12]:
def test(ckpt_path='./log/2023-04-14_16-04-27/checkpoint.pt', device='cuda'):
    dataloaders = get_dataloaders(batch_size=1)
    testDataLoader = dataloaders['val']

    model, _ = get_model_optimizer(ckpt_path=ckpt_path, device=device)
    model.eval()  # Set model to evaluate mode  设置模型为评估模式

    correct = 0
    total = 0
    time = 0
    # since we're not training, we don't need to calculate the gradients for our outputs 因为我们不是在训练，所以我们不需要计算输出的梯度。
    with torch.no_grad():
        for data in testDataLoader:
            images, labels = data
            # calculate outputs by running images through the network 通过网络运行图像来计算输出
            start = timeit.default_timer()
            outputs = model(images)
            stop = timeit.default_timer()
            # the class with the highest energy is what we choose as prediction我们选择能量最高的类作为预测的对象。
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            time += stop - start
            correct += (predicted == labels).sum().item()

            print(f'label:{labels}; pred:{predicted}; out:{outputs} filename:')
            plt.grid(b=False)
            plt.imshow((images[0, 0] + 1) / 2)
            plt.colorbar()
            plt.show()
            plt.close()

        acc = float(correct) / len(testDataLoader.dataset)
        print(f'total num: {total}; acc: {acc}; time: {time / total}s on {device}')

In [13]:
def get_dataloaders(resize=(600, 800), center_crop_size=(448, 448), data_dir='dataset', batch_size=8,num_workers=6):
    #自定义训练集和验证集的transfrom
    data_transforms = {'train': T.Compose(
        [
            # MyCropTransform(),
            T.Resize(resize),
            #T.RandAugment(),
            T.CenterCrop(center_crop_size),
            
            T.Resize((224, 224)),
            
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),#为防止过拟合，可以对数据进行翻转，亮度，对比度等数据增广
        ]),

        'val': T.Compose(
            [
                # MyCropTransform(),
                T.Resize(resize),
                T.CenterCrop(center_crop_size),
                T.Resize((224, 224)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])}

    # Create training and validation dataset and dataloaders 生成训练和验证数据集和数据加载器
    datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}#生成训练集
    dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size, shuffle=True, num_workers=6,pin_memory=False,prefetch_factor=64,persistent_workers=True) for x in
                   ['train', 'val']}#定义dataloaders,并且生成dataloaders
    """
    dataset：即上面自定义的dataset；
    batch_size：一个batch中样本个数；
    shuffle：划分batch前是否打乱顺序；
    sampler：定义抽样的策略；
    batch_sampler：定义批次抽样的策略；
    num_worker：定义多线程方法，默认为0
    """
    return dataloaders

In [22]:
os.getcwd()
resize=(600, 800)
center_crop_size=(448, 448)
data_dir='data'    #自定义训练集和验证集的transfrom
data_transforms = {'train': T.Compose(
        [
            # MyCropTransform(),
            T.Resize(resize),
            #T.RandAugment(),
            T.CenterCrop(center_crop_size),
            
            T.Resize((224, 224)),
            
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),#为防止过拟合，可以对数据进行翻转，亮度，对比度等数据增广
        ]),

        'val': T.Compose(
            [
                # MyCropTransform(),
                T.Resize(resize),
                T.CenterCrop(center_crop_size),
                T.Resize((224, 224)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])}

b = ImageFolder(os.path.join('data', 'train'), data_transforms['train'])
b.classes

['0', '1']

In [63]:
dataloaders = get_dataloaders()#查看
a = iter(dataloaders['train'])
next(a)

[tensor([[[[ 0.4745,  0.4588,  0.4510,  ...,  0.3882,  0.4431,  0.4353],
           [ 0.4980,  0.4980,  0.5059,  ...,  0.4196,  0.4431,  0.3961],
           [ 0.5451,  0.5529,  0.5686,  ...,  0.4196,  0.3882,  0.2941],
           ...,
           [ 0.5059,  0.4196,  0.3098,  ...,  0.4824,  0.4118,  0.3333],
           [ 0.4510,  0.3647,  0.2706,  ...,  0.4745,  0.3804,  0.2784],
           [ 0.3725,  0.3020,  0.2314,  ...,  0.4667,  0.3882,  0.2941]],
 
          [[ 0.0510,  0.0353,  0.0275,  ..., -0.0196,  0.0353,  0.0353],
           [ 0.0745,  0.0745,  0.0824,  ...,  0.0118,  0.0353, -0.0039],
           [ 0.1216,  0.1294,  0.1451,  ...,  0.0118, -0.0196, -0.1059],
           ...,
           [ 0.0588, -0.0353, -0.1373,  ...,  0.0118, -0.0667, -0.1686],
           [-0.0039, -0.0902, -0.1765,  ..., -0.0118, -0.1059, -0.2235],
           [-0.0824, -0.1529, -0.2157,  ..., -0.0196, -0.0980, -0.2078]],
 
          [[ 0.5373,  0.5216,  0.5137,  ...,  0.5686,  0.6235,  0.6314],
           [ 

In [14]:
def get_model_optimizer(num_classes=2, lr=0.0001, ckpt_path=None, device='cuda'):
    # '/home/congliu/linatech/huangsisheng/log/2022-03-30_19-18-39/checkpoint.pt'

    #model = models.efficientnet_b6(pretrained=True)
    #model = models.regnet_x_400mf(pretrained=True)  # 2022-04-27_15-36-17  SGD acc=0.64 adam
    #model = models.regnet_x_1_6gf(pretrained=True)  # 2022-04-27_22-27-11 acc=0.61  2022-10-22_09-08-27 acc=96.82
    #model = models.regnet_x_8gf(pretrained=True)  # 2022-04-28_20-03-08     acc=0.59
    #model=models.regnet_y_3_2gf(pretrained=True)#2022-05-16_18-44-35 acc=0.81
    # model = models.regnet_y_3_2gf(pretrained=True) #2022-05-18_12-44-59 SGD优化器 acc=0.81 adam优化器acc=0.87  ac=0.887  adam优化器acc=0.87  ac=0.887
    #model = models.resnet50(pretrained=True)#2022-05-18_18-02-39 adam优化器acc=0.86
    #model = models.resnet50(pretrained=True)  # 2022-04-27_00-36-54  SGD acc=0.67   Adam acc=97.54 2022-10-21_22-59-02
    model = ACmix_ResNet()
    #set_parameter_requires_grad(model, feature_extracting=True)#冻结卷积层
    model.fc= nn.Linear(model.fc.in_features, num_classes)#替换fc层
    #model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    if ckpt_path != None:
        model = load_model(ckpt_path, model, device)
    #params_to_update = model.parameters(
    #params_to_update = []
    print("Params to learn:")
    # for name, param in model.named_parameters():#只训练最后的全连接层，做法如下。
    #      if name not in ["fc.weight", "fc.bias"]:
    #         param.requires_grad = False  # 关键一步，设置为False之后，优化器溜不会对参数进行更新,特征层中参数都固定住，不会发生梯度的更新
    #         #是修改了最后的一层全连接层，对于该层之外的全部层可以冻结其参数，只训练最后的全连接层
    params_1x = [param for name, param in model.named_parameters()
                 if name not in ["fc.weight", "fc.bias"]]
    # 由于这里我们只需要对最后一层修改后的全连接层进行训练，而对其他层不需要怎么训练，根据这种特性，可以分为两个参数组丢进优化器中。
    optimizer = optim.Adam([{'params': params_1x}, {'params': model.fc.parameters(), 'lr': lr * 10}], lr=lr,weight_decay=0.001)

    return model, optimizer


In [15]:
def main(num_epochs=200):
    # dataloaders 数据加载
    dataloaders = get_dataloaders()
    # model optimizer  模型优化器
    model, optimizer = get_model_optimizer()
    # Train and evaluate
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()  # Setup the loss fxn  设置损失fxn
    model, hist = train_model(model, dataloaders, criterion, optimizer, device, num_epochs=num_epochs)
    save_model(model, hist)
    

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()#类似于结构体
    parser.add_argument("-tn", "--train", help="train model", action="store_true",default=True)
    parser.add_argument("-ts", "--test", help="test model", action="store_true",default=False)
    args = parser.parse_args(args=[])
    # main()
    if args.train:
        torch.cuda.empty_cache()
        main()
    elif args.test:
        test()

Params to learn:
Epoch 0/199
----------
train Loss: 0.5475 Acc: 0.7454
val Loss: 0.4626 Acc: 0.8078

Epoch 1/199
----------
train Loss: 0.4418 Acc: 0.8011
val Loss: 0.3660 Acc: 0.8497

Epoch 2/199
----------
train Loss: 0.3577 Acc: 0.8471
val Loss: 0.2701 Acc: 0.8965

Epoch 3/199
----------
train Loss: 0.2771 Acc: 0.8871
val Loss: 0.2105 Acc: 0.9179

Epoch 4/199
----------
train Loss: 0.2323 Acc: 0.9078
val Loss: 0.1979 Acc: 0.9255

Epoch 5/199
----------


In [45]:
 def alter_fileName(path): # 获取目录下所有jpg格式文件的文件名
    n=0
    filelist= os.listdir(targe_path)
    for i in filelist:  #i是path目录下的文件名
        oldname=targe_path+os.sep+filelist[n]#oldname是原文件路径
        newname=targe_path+os.sep+str(n+1)+'.jpeg'#newname是新文件名，路径+新文件名
        os.rename(oldname,newname) #改名字
        print(oldname,'======>',newname)
        n+=1
path='./dataset/val/1'
targe_path='./dataset/val/1'
alter_fileName(targe_path)

./dataset/val/1/2.jpeg ======> ./dataset/val/1/1.jpeg
./dataset/val/1/3.jpeg ======> ./dataset/val/1/2.jpeg
./dataset/val/1/4.jpeg ======> ./dataset/val/1/3.jpeg
./dataset/val/1/5.jpeg ======> ./dataset/val/1/4.jpeg
./dataset/val/1/6.jpeg ======> ./dataset/val/1/5.jpeg
./dataset/val/1/7.jpeg ======> ./dataset/val/1/6.jpeg
./dataset/val/1/8.jpeg ======> ./dataset/val/1/7.jpeg
./dataset/val/1/9.jpeg ======> ./dataset/val/1/8.jpeg
./dataset/val/1/10.jpeg ======> ./dataset/val/1/9.jpeg
./dataset/val/1/11.jpeg ======> ./dataset/val/1/10.jpeg
./dataset/val/1/12.jpeg ======> ./dataset/val/1/11.jpeg
./dataset/val/1/13.jpeg ======> ./dataset/val/1/12.jpeg
./dataset/val/1/14.jpeg ======> ./dataset/val/1/13.jpeg
./dataset/val/1/15.jpeg ======> ./dataset/val/1/14.jpeg
./dataset/val/1/16.jpeg ======> ./dataset/val/1/15.jpeg
./dataset/val/1/17.jpeg ======> ./dataset/val/1/16.jpeg
./dataset/val/1/18.jpeg ======> ./dataset/val/1/17.jpeg
./dataset/val/1/19.jpeg ======> ./dataset/val/1/18.jpeg
./dataset